In [1]:
import sys
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
sys.path.append(".")
# os.environ["MAX_JOBS"] = "100"

import torch
torch.set_printoptions(linewidth=200, threshold=100000)
torch.cuda.is_available()

True

In [2]:
from src.vlstm_fwbw_v1.interface import vlstm_fwbw_torch_obw, vlstm_fwbw_cuda
from src.vlstm_fwbw_v1.interface import vlstm_fwbw_torch_autogradbw

INCLUDE: ['/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/torch/csrc/api/include', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/TH', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/THC', '/home/max/miniconda3/envs/xlstmpt220cu121/include']
/home/max/myrepos/vlstm_cuda/src
/home/max/cpplibs/libtorch/lib:/usr/local/cuda-12.3/lib64:
/home/max/miniconda3/envs/xlstmpt220cu121/lib


Using /home/max/.cache/torch_extensions/py311_cu121 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/max/.cache/torch_extensions/py311_cu121/vlstm_fwbw_v1/build.ninja...
Building extension module vlstm_fwbw_v1...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module vlstm_fwbw_v1...


## CUDA vLSTM forward backward

In [3]:
S = 8 #32 #16 #8 # seq len
B = 1 # batch size
NH = 1 # num heads
DH = 8 # dim per head
DTYPE = torch.bfloat16 #float32
DEVICE = torch.device("cuda:0")

In [4]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
# fixed:
# qs = torch.arange((B*NH*S*DH), device=DEVICE, dtype=DTYPE).reshape((B, NH, S, DH)) / 10.
# ks = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
# vs = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.

# random: 
qs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ks = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
vs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
# igs = (1. + torch.arange((B * NH * S), device=DEVICE, dtype=DTYPE)).reshape(B, NH, S, 1) / 10.
# igs = torch.zeros((B, NH, S, 1), device=DEVICE, dtype=DTYPE) #/ 10.
igs = torch.randn((B, NH, S, 1), device=DEVICE, dtype=DTYPE) #/ 10.
# fgs = torch.ones((B, NH, S, 1), device=DEVICE, dtype=DTYPE)
fgs = torch.randn((B, NH, S, 1), device=DEVICE, dtype=DTYPE)

# dHs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)

### match through autograd

In [5]:
# now we need to apply the groupnorm directly afterwards
from vlstm_formulations.ln import MultiHeadLayerNorm

mh_layernorm = MultiHeadLayerNorm(ndim=NH*DH, eps=1e-6).to(device=DEVICE, dtype=DTYPE)
mh_layernorm.weight, mh_layernorm.bias

(Parameter containing:
 tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0', dtype=torch.bfloat16, requires_grad=True),
 None)

In [6]:
offset = 3.* torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)

In [7]:
fgs_pt_ag = fgs.clone().detach().requires_grad_(True)
igs_pt_ag = igs.clone().detach().requires_grad_(True)
qs_pt_ag = qs.clone().detach().requires_grad_(True)
ks_pt_ag = ks.clone().detach().requires_grad_(True)
vs_pt_ag = vs.clone().detach().requires_grad_(True)

In [8]:
# pytorch obw
hs_pt_ag, n_pt_ag, m_pt_ag = vlstm_fwbw_torch_obw( #vlstm_fwbw_torch_autogradbw( #vlstm_fwbw_torch_obw(
    queries=qs_pt_ag,
    keys=ks_pt_ag,
    values=vs_pt_ag,
    igate_preact=igs_pt_ag,
    fgate_preact=fgs_pt_ag,
)
hs_pt_ag_scaled = mh_layernorm(hs_pt_ag)
# hs_pt_ag_scaled.sum().backward()
((hs_pt_ag_scaled+offset)**2).sum().backward()

In [9]:
fgs_cu_ag = fgs.clone().detach().requires_grad_(True)
igs_cu_ag = igs.clone().detach().requires_grad_(True)
qs_cu_ag = qs.clone().detach().requires_grad_(True)
ks_cu_ag = ks.clone().detach().requires_grad_(True)
vs_cu_ag = vs.clone().detach().requires_grad_(True)

In [10]:
# pytorch autograd
# hs_cu_ag, n_cu_ag, m_cu_ag = vlstm_fwbw_torch_autogradbw(
#     queries=qs_cu_ag,
#     keys=ks_cu_ag,
#     values=vs_cu_ag,
#     igate_preact=igs_cu_ag,
#     fgate_preact=fgs_cu_ag,
# )
# hs_cu_ag_scaled = mh_layernorm(hs_cu_ag)
# hs_cu_ag_scaled.sum().backward()

In [11]:
# cuda kernel
hs_cu_ag, n_cu_ag, m_cu_ag, matD_cu_ag = vlstm_fwbw_cuda(
    mat_Q=qs_cu_ag,
    mat_K=ks_cu_ag,
    mat_V=vs_cu_ag,
    vec_igp=igs_cu_ag,
    vec_fgp=fgs_cu_ag,
)
hs_cu_ag_scaled = mh_layernorm(hs_cu_ag)
# hs_cu_ag_scaled.sum().backward()
((hs_cu_ag_scaled+offset)**2).sum().backward()

before kernel dispatch - bfloat16!
B: 1, NH: 1, S: 8, DH: 8
blocksxy: 1-2, threadsxy: 4-4, shared_mem in bytes: 2976
In FW-Kernel: gdim.x: 1, gdim.y: 2, gdim.z: 1, bdim.x: 4, bdim.y: 4
In FW-Kernel: QtileDim: 8, KVtileDim: 8, TblockDim:4
before kernel dispatch - bfloat16!
B: 1, NH: 1, S: 8, DH: 8
blocksxy: 1-1, threadsxy: 4-4, shared_mem in bytes: 3968
In BW-Kernel: gdim.x: 1, gdim.y: 1, gdim.z: 1, bdim.x: 4, bdim.y: 4
In BW-Kernel: QtileDim: 8, KVtileDim: 8, TblockDim:4


In [12]:
FW_RTOL = 1e-10
FW_ATOL = 1e-4
BW_RTOL = FW_ATOL
BW_ATOL = FW_ATOL
print(f"fw hs match: {torch.allclose(hs_cu_ag, hs_pt_ag, rtol=FW_RTOL, atol=FW_ATOL)}")
print(f"fw hs scaled match: {torch.allclose(hs_cu_ag_scaled, hs_pt_ag_scaled, rtol=FW_RTOL, atol=FW_ATOL)}")
print(f"fw n match: {torch.allclose(n_cu_ag, n_pt_ag, rtol=FW_RTOL, atol=FW_ATOL)}")
print(f"fw m match: {torch.allclose(m_cu_ag, m_pt_ag, rtol=FW_RTOL, atol=FW_ATOL)}")
# print(f"fw D match: {torch.allclose((matD_cu_ag - matD_pt_ag).tril(), torch.zeros_like((matD_cu)), rtol=FW_RTOL, atol=FW_ATOL)}")

print(f"delta Q match: {torch.allclose(qs_cu_ag.grad, qs_pt_ag.grad, rtol=BW_RTOL, atol=BW_ATOL)}")
print(f"delta K match: {torch.allclose(ks_cu_ag.grad, ks_pt_ag.grad, rtol=BW_RTOL, atol=BW_ATOL)}")
print(f"delta V match: {torch.allclose(vs_cu_ag.grad, vs_pt_ag.grad, rtol=BW_RTOL, atol=BW_ATOL)}")
print(f"delta Igate match: {torch.allclose(igs_cu_ag.grad, igs_pt_ag.grad, rtol=BW_RTOL, atol=BW_ATOL)}")
print(f"delta Fgate match: {torch.allclose(fgs_cu_ag.grad, fgs_pt_ag.grad, rtol=BW_RTOL, atol=BW_ATOL)}")

fw hs match: False
fw hs scaled match: False
fw n match: False
fw m match: False
delta Q match: False
delta K match: False
delta V match: False
delta Igate match: False
delta Fgate match: False


In [13]:
m_cu_ag, m_pt_ag

(tensor([[[[ 0.3516],
           [-0.0742],
           [-0.5195],
           [ 0.3105],
           [-0.3652],
           [ 0.7266],
           [ 0.1641],
           [ 0.4121]]]], device='cuda:0', dtype=torch.bfloat16, grad_fn=<vLSTMParallelFwBwCudaBackward>),
 tensor([[[[ 0.3516],
           [-0.0781],
           [-0.5234],
           [ 0.3105],
           [-0.3613],
           [ 0.7266],
           [ 0.1641],
           [ 0.4121]]]], device='cuda:0', dtype=torch.bfloat16, grad_fn=<vLSTMParallelFwBwWithGroupNormBackward>))

In [19]:
qs_cu_ag.grad - qs_pt_ag.grad

tensor([[[[ 1.1749e-03, -3.5706e-03,  5.9814e-03, -4.7607e-03,  8.9111e-03,  5.6152e-03,  8.3923e-04, -6.0425e-03],
          [ 0.0000e+00,  0.0000e+00, -7.8125e-03, -1.4648e-03, -2.4414e-03, -1.9531e-03,  0.0000e+00, -9.7656e-04],
          [-7.8125e-03,  6.2500e-02, -1.2500e-01,  6.2500e-02,  0.0000e+00, -6.2500e-02,  0.0000e+00,  0.0000e+00],
          [-7.8125e-03, -6.2500e-02,  9.3750e-02, -6.2500e-02,  7.8125e-02,  5.4688e-02,  1.0352e-01, -7.8125e-03],
          [-1.0156e-01, -5.0000e-01,  6.5625e-01, -4.3750e-01,  4.3750e-01,  3.4375e-01,  8.7500e-01,  3.1250e-02],
          [ 1.5625e-01,  6.6406e-02, -1.2500e-01,  8.4961e-02, -6.8750e-01, -4.6875e-02, -1.3965e-01, -1.5625e-01],
          [-3.1250e-02,  0.0000e+00, -1.5625e-02, -5.8594e-03,  1.0938e-01,  0.0000e+00, -1.5625e-02,  7.8125e-03],
          [-1.2500e-01, -6.2500e-02,  2.5000e-01, -1.5625e-02, -5.0000e-01,  2.5000e-01,  2.5000e-01, -3.9062e-02]]]], device='cuda:0', dtype=torch.bfloat16)

In [20]:
torch.abs(qs_cu_ag.grad - qs_pt_ag.grad).max()

tensor(0.8750, device='cuda:0', dtype=torch.bfloat16)

In [21]:
torch.abs(ks_cu_ag.grad - ks_pt_ag.grad).max()

tensor(0.7500, device='cuda:0', dtype=torch.bfloat16)

In [22]:
torch.abs(ks_cu_ag.grad - ks_pt_ag.grad).max()

tensor(0.7500, device='cuda:0', dtype=torch.bfloat16)

In [18]:
torch.abs(vs_cu_ag.grad - vs_pt_ag.grad).max()

tensor(0.3125, device='cuda:0', dtype=torch.bfloat16)